In [14]:
import functools
from functools import reduce
from functools import partial
import sys
import queue
import re
import operator
import time


ops = {
       '<': operator.lt,
       '<=': operator.le,
       '>': operator.gt,
       '>=': operator.ge,
       }
def foc(x, value):
    return x == value


class ConstraintVar:
    def __init__(self, d, n):
        self.domain = [v for v in d]
        self.name = n


class UnaryConstraint:
    def __init__(self, v, fn):
        self.var = v
        self.func = fn


class BinaryConstraint:
    def __init__(self, v1, v2, fn):
        self.var1 = v1
        self.var2 = v2
        self.func = fn

def all_Diff(constraints, v):
    fn = lambda x, y: x != y
    for i in range(len(v)):
        for j in range(len(v)):
            if (i != j):
                constraints.append(BinaryConstraint(v[i], v[j], fn))

def make_Futoshiki(caseNum):

        uconstraints = []
        bconstraints = []

        lines = open('Futoshiki_matrix.txt').readlines()
        testLine = caseNum 
        l = lines[testLine]
        l = re.sub('[ ]','',l)

        n = eval(re.findall('^\d+',l)[0])
        l = re.sub('^\d+','',l)

        domain = []
        for i in range(n):
                domain.append(i+1)

        rows = []
        cols = []
        variables = dict()
        for i in range (n):
                rows.append(chr(ord('A') + i))
                cols.append(chr(ord('1') + i))
        varNames = [ x+y for x in rows for y in cols ]

        for varname in varNames:
                variables[varname] = ConstraintVar( domain,varname )

        cs=re.findall('\w+\W+\w+',l)

        for c in cs:
                if re.findall('\w+\d+<\w+\d+',c) or re.findall('\w+\d+>\w+\d+',c):
                        lvar = re.findall('^\w+\d+',c)[0]
                        rvar = re.findall('\w+\d+$',c)[0]
                        op = re.findall('\W',c)[0]
                        if op == '>':
                            fn = lambda x,y: x > y
                            fn1 = lambda x,y: x < y
                        elif op == '<':
                            fn = lambda x,y: x < y
                            fn1 = lambda x,y: x > y
                        bconstraints.append(BinaryConstraint(variables[lvar], variables[rvar], fn))
                        bconstraints.append(BinaryConstraint(variables[rvar], variables[lvar], fn1))
                else:
                        if re.findall('\w+\d+=\d+',c):
                                var = re.findall('^\w+\d+',c)[0]
                                value = re.findall('\d+$',c)[0]
                        elif re.findall('\d+=\w+\d+',c):
                                var = re.findall('\w+\d+$',c)[0]
                                value = re.findall('^\d+$',c)[0]
                        fn = functools.partial(foc, value = int(value))
                        '''print("*****************var")
                        print(variables[var].name, value)
                        print('var,val,fn(1) ',var,'==',value,fn(1))'''
                        uconstraints.append(UnaryConstraint(variables[var], fn))

        for r in rows:
            aRow = []
            for k in variables.keys():
                if (str(k).startswith(r)):
                    aRow.append(variables[k])
            all_Diff(bconstraints, aRow)

        for c in cols:
            aCol = []
            for k in variables.keys():
                key = str(k)
                if (key[1] == c):
                    aCol.append(variables[k])
            all_Diff(bconstraints, aCol)
        return variables, n, uconstraints, bconstraints 

def Rev(bc):

    deleted = []
    if isinstance(bc, UnaryConstraint):
        dom = list(bc.var.domain)
        for x in dom:
            if not bc.func(x):
                bc.var.domain.remove(x)
        return True
    elif isinstance(bc, BinaryConstraint):
        dom1 = list(bc.var1.domain)
        dom2 = list(bc.var2.domain)

        for x in dom1:
            needtoremove = True
            for y in dom2:
                if x == y:
                    continue
                if True == bc.func(x, y):
                    needtoremove = False
                    break
            if needtoremove:
                deleted.append(x)
                bc.var1.domain.remove(x)
        return deleted

def AC_3(variables, constraints):

    for u in constraints[0]:
        nodeConsistent(u)

    worklist = queue.Queue()
    for i in constraints[1]:
        worklist.put(i)

    counter = 0

    while not worklist.empty():
        counter += 1
        constraint = worklist.get()
        deleted = Rev(constraint)
        if deleted:
            for i in constraints[1]:
                if i.var2.name == constraint.var1.name:
                    worklist.put(i)
                    
    return counter

def node_Cons(uc):
    domain = list(uc.var.domain)
    for x in domain:
        if not uc.func(x):
            uc.var.domain.remove(x)


def print_dom(vars, n):
    count = 0
    for k in sorted(vars.keys()):
        print(k, '{', vars[k].domain, '}, ', end="")
        count = count + 1
        if (0 == count % n):
            print(' ')

def print_asn(assignment, n):
    if not assignment:
        print("Problem not consistent")
        return
    count = 0
    for k in sorted(assignment.keys()):
        print(k, '{', assignment[k], '}, ', end="")
        count += 1
        if 0 == count % n:
            print(' ')

def runFutoshiki(caseNum):

    print("Test-case %d"%(caseNum+1))
    variables, n , uconstraints, bconstraints = make_Futoshiki(caseNum )
    cons = [uconstraints, bconstraints]
    t1 = time.clock()
    counter = AC_3(variables, cons)
    t2 = time.clock()
    print("\nAC-3 finished, executed for", counter, "times")
    print("Time: %.4f seconds" % (t2 - t1))
    printDomains(variables, n)
    print("-----------------X-----------------")

if __name__ == '__main__':
    for i in range(4):
        runFutoshiki(i)

Test-case 1

AC-3 finished, executed for 221 times
Time: 0.0017 seconds
A1 { [1, 2, 4] }, A2 { [1, 2] }, A3 { [3] }, A4 { [2, 4] },  
B1 { [1, 2] }, B2 { [1, 2, 3, 4] }, B3 { [1, 2, 4] }, B4 { [1, 2, 3] },  
C1 { [2, 3] }, C2 { [1, 2] }, C3 { [1, 2, 4] }, C4 { [1, 2, 3, 4] },  
D1 { [3, 4] }, D2 { [1, 2, 3, 4] }, D3 { [1, 2] }, D4 { [2, 3, 4] },  
-----------------X-----------------
Test-case 2

AC-3 finished, executed for 514 times
Time: 0.0042 seconds
A1 { [2, 3, 5] }, A2 { [1, 2] }, A3 { [2, 3] }, A4 { [4] }, A5 { [1, 2, 5] },  
B1 { [1, 2, 3] }, B2 { [4] }, B3 { [1, 2, 3, 5] }, B4 { [1, 2, 3, 5] }, B5 { [1, 2, 5] },  
C1 { [1, 2, 3, 4, 5] }, C2 { [1, 2, 3, 5] }, C3 { [3, 4, 5] }, C4 { [1, 2, 3, 5] }, C5 { [1, 2, 4, 5] },  
D1 { [1, 2, 3, 4, 5] }, D2 { [1, 2, 3, 5] }, D3 { [2, 3, 4] }, D4 { [1, 2, 3] }, D5 { [1, 2, 4, 5] },  
E1 { [1, 2, 4, 5] }, E2 { [1, 2] }, E3 { [2, 4, 5] }, E4 { [1, 2] }, E5 { [3] },  
-----------------X-----------------
Test-case 3

AC-3 finished, executed for